In [1]:
from osgeo import gdal, osr
import numpy as np

ModuleNotFoundError: No module named 'osgeo'

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Daily to monthly transform

By default, data is written with float32 dtype. It works well with future climate ("NASA/NEX-GDDP" from GEE)

In [13]:
def array2raster(newRasterfn, dataset, array, metadata, dtype='Float32'):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform()

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte":
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    elif dtype == "Int16":
        GDT_dtype = gdal.GDT_Int16
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)

        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
            outband.SetDescription(metadata)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [20]:
path = 'drive/MyDrive/Geo_data/Crop_Asia/Climate_future/'
model_scenario = 'MPI-ESM-MR_rcp45_'
monthes = np.arange(1,13)

In [21]:
# Compose monthly tiffs from Future Climate CMIP5 daily Data. Works with Google Drive (i.e. Colab)

#Reference geo tiff to get geoinformation from
year = '2027'
source = gdal.Open(path + model_scenario + 'tasmax_'+ year + '_1.tif')

# Numbers for specific month
for month in monthes:
    print('month', month)

    # Count number of bands (= number of days) in current month
    out = gdal.Open(path + model_scenario + 'pr_'+year+'_' + str(month) + '.tif')
    n_bands = out.RasterCount
    w = out.RasterXSize
    h = out.RasterYSize
    tmax_daily, tmin_daily, pr_daily = np.empty((h,w,0)), np.empty((h,w,0)), np.empty((h,w,0))
    out = None

    for i in range(n_bands):
        tmmx = gdal.Open(path + model_scenario + 'tasmax_'+year+'_' + str(month) + '.tif')
        raster_tmmx = tmmx.GetRasterBand(i+1).ReadAsArray(0, 0, tmmx.RasterXSize, tmmx.RasterYSize)
        tmax_daily = np.dstack((tmax_daily, raster_tmmx))

        tmmn = gdal.Open(path + model_scenario + 'tasmin_'+year+'_' + str(month) + '.tif')
        raster_tmmn = tmmn.GetRasterBand(i+1).ReadAsArray(0, 0, tmmn.RasterXSize, tmmn.RasterYSize)
        tmin_daily = np.dstack((tmin_daily, raster_tmmn))

        pr = gdal.Open(path + model_scenario + 'pr_'+year+'_' + str(month) + '.tif')
        raster_pr = pr.GetRasterBand(i+1).ReadAsArray(0, 0 ,pr.RasterXSize, pr.RasterYSize)
        pr_daily = np.dstack((pr_daily, raster_pr))

    tmax_monthly = np.mean( tmax_daily, axis=2 ) - 273.15  # convert Kelvins into Celcius
    tmax_monthly = tmax_monthly*10  # coincide with TerraClimate scale factor

    tmin_monthly = np.mean( tmin_daily, axis=2 )- 273.15  # convert Kelvins into Celcius
    tmin_monthly = tmin_monthly*10  # coincide with TerraClimate scale factor

    pr_monthly = np.sum ( pr_daily, axis=2 )*60*60*24  #convert kg/m2*s into mm/day

    # This part rewrites file if it already exists
    array2raster(path + model_scenario + 'tmmx_'+year+'_'+ str(month)+ '_avg.tif',
                 source, tmax_monthly,
                 model_scenario + str(year) + str(month)+'tasmax_avg')
    array2raster(path + model_scenario + 'tmmn_'+year+'_'+ str(month)+ '_avg.tif',
                 source, tmin_monthly,
                 model_scenario + str(year) + str(month) +'tasmin_avg')
    array2raster(path + model_scenario + 'pr_'+year+'_'+ str(month)+ '_avg.tif',
                 source, pr_monthly,
                 model_scenario + str(year) + str(month) +'pr_avg')

# Close raster
tmmx = None
tmin = None
pr = None

month 1
month 2
month 3
month 4
month 5
month 6
month 7
month 8
month 9
month 10
month 11
month 12


In [23]:
import os
len(os.listdir(path))

432

In [24]:
z=[]

In [25]:
for file in os.listdir(path):
  if 'avg' not in file:
    os.remove(os.path.join(path, file))
    # print(file)
print(i)

30


In [ ]:
import os

In [ ]:
len(z)

108